In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0


In [3]:
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu123

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu123
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 24.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3511090 sha256=b757714740a39dfcca2be756cc61818a25d0e0b6ce8f43686ebef515ec5d282e
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python


In [4]:
from llama_cpp import Llama
import tqdm

llm = Llama.from_pretrained(
	repo_id="QuantFactory/Meta-Llama-3-8B-Instruct-GGUF",
	filename="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
    n_ctx=4096,
)

questions = [
    """Here's the idea: "{}"
What genre(s) could this story fall into? Answer in 1 sentence.
""",
    "What kind of message do you want the story to have? Answer in 1 sentence.",
    "Are there any sub-themes you want to incorporate? Answer in 1 sentence.",
    "Summarize the plot of the story for me in a single line."
    "In what time period does your story take place? Answer in 1 sentence.",
    "What's the primary setting or location? Answer in 1 sentence.",
    "Are there any unique aspects of this world that affect the story? Answer in 1 sentence.",
    "Are there any social, political, or technological elements that are important? Answer in 1 sentence.",
    "Who is your protagonist? What do they want?",
    "What obstacles stand in their way?",
    "Who are the other key characters?",
    "What are the relationships between characters?",
    "What motivates each character's actions?",
    "What flaws or quirks make your characters interesting?",
    "What inciting incident sets the story in motion? Answer in 1 sentence.",
    "What are the key events or turning points?",
    "What is the main conflict (internal, external, or both)?",
    "How will the tension escalate?",
    "What's at stake for the characters?",
    "How might the story end? What changes for the characters?",
    "Which scene or moment will you focus on?",
    "What context can you imply rather than state directly?",
    "Which details are essential to include?",
    "What sensory elements can quickly establish the setting?",
    """Make me an outline for making a flash fiction story using all of this information. 
This story will have a maximum word limit of 700-1000 words; so you would only be able to focus on the one scene. Having said that, break this scene up into beats for the outline. 
Do not plan for sudden jumps in time/location between these beats, unless absolutely necessary.

Add some details about the characters, and the setting of this scene at the end. Use a maximum of 500 words.

""",
    "Write a story of between 700-1000 words using this outline."
]



def make_story(idea):
    qs = list(questions)
    qs[0] = qs[0].format(idea)
    
    messages = [
        {'role': 'system', 'content': 'Over the next couple of messages, I want you to build up elements for a story (flash fiction) - which you will later use to make an outline. Use a maximum of 5 sentences unless instructed otherwise.'},
    ]

    
    for q in tqdm.tqdm(qs):
        messages.append({'role': 'user', 'content': q})
        print(messages[-1])
        output_message = llm.create_chat_completion(messages)['choices'][0]['message']
        print(output_message)
        messages.append(output_message)

    return messages, output_message['content']

Meta-Llama-3-8B-Instruct.Q4_0.gguf:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--QuantFactory--Meta-Llama-3-8B-Instruct-GGUF/snapshots/86e0c07efa3f1b6f06ea13e31b1e930dce865ae4/./Meta-Llama-3-8B-Instruct.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...


In [40]:
messages, story = make_story('I\'ve had enough.')

print(story)

  0%|          | 0/25 [00:00<?, ?it/s]

{'role': 'user', 'content': 'Here\'s the idea: "I\'ve had enough."\nWhat genre(s) could this story fall into? Answer in 1 sentence.\n'}


llama_perf_context_print:        load time =   10588.50 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    85 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    37 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   25621.55 ms /   122 tokens
  4%|▍         | 1/25 [00:25<10:15, 25.63s/it]Llama.generate: 122 prefix-match hit, remaining 28 prompt tokens to eval


{'role': 'assistant', 'content': 'This phrase could fall into the genres of Science Fiction, Mystery, or Psychological Thriller, as it suggests a protagonist who has reached a breaking point and is prepared to take drastic action.'}
{'role': 'user', 'content': 'What kind of message do you want the story to have? Answer in 1 sentence.'}


llama_perf_context_print:        load time =   10588.50 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    28 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    35 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   17428.06 ms /    63 tokens
  8%|▊         | 2/25 [00:43<07:58, 20.81s/it]Llama.generate: 185 prefix-match hit, remaining 27 prompt tokens to eval


...

{'role': 'assistant', 'content': "To quickly establish the setting, I can use sensory elements such as the protagonist's sense of dusty, stale air filling their lungs, the faint smell of decay and neglect wafting from the abandoned buildings, and the sound of distant, muffled sirens echoing through the streets, evoking a sense of unease and foreboding."}
{'role': 'user', 'content': 'Make me an outline for making a flash fiction story using all of this information. \nThis story will have a maximum word limit of 700-1000 words; so you would only be able to focus on the one scene. Having said that, break this scene up into beats for the outline. \nDo not plan for sudden jumps in time/location between these beats, unless absolutely necessary.\n\nAdd some details about the characters, and the setting of this scene at the end. Use a maximum of 500 words.\n\n'}


llama_perf_context_print:        load time =   10588.50 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   113 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   621 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  280645.59 ms /   734 tokens
 96%|█████████▌| 24/25 [14:04<01:43, 103.46s/it]Llama.generate: 2496 prefix-match hit, remaining 25 prompt tokens to eval


{'role': 'assistant', 'content': '**Flash Fiction Outline: "I\'ve Had Enough"**\n\n**Scene:** The protagonist, JESSICA, stands outside the town\'s central square, where the oppressive leader, MAYOR HARRISON, is giving a speech to the townspeople.\n\n**Beat 1: Setup**\n\n* Introduce Jessica, a young adult who\'s been struggling to make a living in this small, isolated town.\n* Show her walking into the central square, where she\'s surrounded by the town\'s residents, all staring at Mayor Harrison.\n* Establish the oppressive atmosphere, with hints at the town\'s dark history and the protagonist\'s sense of isolation and disconnection.\n\n**Beat 2: The Inciting Incident**\n\n* Mayor Harrison begins his speech, boasting about the town\'s "progress" and "prosperity."\n* Jessica, fed up with the mayor\'s empty promises and the town\'s stagnant state, decides to take a stand.\n* She pushes her way to the front, her eyes locked on the mayor, and begins to shout her dissent.\n\n**Beat 3: The C

llama_perf_context_print:        load time =   10588.50 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    25 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   733 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  327403.78 ms /   758 tokens
100%|██████████| 25/25 [19:31<00:00, 46.86s/it] 

{'role': 'assistant', 'content': '"I\'ve Had Enough"\n\nThe sun beat down on the town\'s central square, casting a harsh light on the dusty, worn-down buildings that lined the streets. The air was heavy with the scent of decay and neglect, a constant reminder of the town\'s stagnant state. Jessica, a young adult with a fiery spirit, walked into the square, her eyes scanning the crowd of townspeople gathered to hear Mayor Harrison\'s speech.\n\nJessica had grown tired of the mayor\'s empty promises and the town\'s lack of progress. She was sick of hearing the same tired phrases, day in and day out, as the mayor boasted about the town\'s "prosperity" and "growth." But today, something was different. Today, Jessica had had enough.\n\nAs she pushed her way to the front of the crowd, Mayor Harrison\'s voice boomed out, touting the town\'s achievements and declaring that this was the best year yet. But Jessica wasn\'t buying it. She had seen the same promises made and broken time and time ag

In [5]:
def prompt_story_direct(idea):
    prompt = f"Write me a story around 700-1000 words based on the idea: '{idea}'"
    
    messages = [{'role':'user', 'content': prompt}]
    output_message = llm.create_chat_completion(messages)['choices'][0]['message']
    
    return output_message, output_message['content']

In [6]:
_, story = prompt_story_direct('I\'ve had enough.')

print(story)

llama_perf_context_print:        load time =    4420.56 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    32 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   852 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  358674.21 ms /   884 tokens


Emily woke up to the sound of her alarm blaring in her ear. She groggily reached over to turn it off, her mind still foggy from the long night she had ahead of her. She worked the night shift at the hospital, and it was a never-ending cycle of exhaustion and fatigue. She rubbed her tired eyes, wondering how she was going to make it through another day.

As she got dressed and headed out the door, Emily couldn't help but think about how her life had become so monotonous. She was 28 years old, and it felt like she was stuck in a rut. Her job was unfulfilling, her social life was non-existent, and her relationship with her boyfriend was strained. She felt like she was just going through the motions every day, without any sense of purpose or fulfillment.

When she arrived at the hospital, Emily was greeted by the usual chaos and commotion. She clocked in, and began her rounds, checking in on her patients and taking their vitals. She was a skilled and dedicated nurse, but even her passion a